# Segmenting and Clustering Neighborhoods in Toronto

Scrapping Toronto neighborhood data from wikipedia which will be utilized for the project

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from urllib.request import urlopen
import requests

Reading the wikipedia page into python

In [2]:
d = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(d, 'lxml')

Filtering the text in html format and converting the table table into data frame

In [3]:
column_names = ['Postalcode','Borough','Neighbourhood']
toronto = pd.DataFrame(columns = column_names)

content = soup.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighbourhood': neighborhood},ignore_index=True)

Filtering out the data to ignore the rows where borough that is 'Not assigned'.

In [4]:
toronto.head()

,Postalcode,Borough,Neighbourhood
0,0,0,0
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Modifying the values in Neighbourhood to values in Borough if the Neighbourhood value is 'Not Assigned' but, Borough has a value.

In [5]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

Grouping Neighbourhood under same Postal Code

In [6]:
df = toronto.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [7]:
df.shape

(103, 3)